In [1]:
# test get_features_from_tsfresh
from inputs import Data, get_features_from_tsfresh,load_separators

data = Data(movingWindow=100,nEventsPerStat=1000)

In [2]:
import h5py
hdf5_directory = 'D:/SDC/py/HDF5/'
filename_raw = hdf5_directory+'tradeinfo.hdf5'
separators_directory = hdf5_directory+'separators/'
f_raw = h5py.File(filename_raw,'r')

In [3]:
thisAsset = data.AllAssets[str(1)]
print(str(0)+". "+thisAsset)
    
group_raw = f_raw[thisAsset]
# load separators
separators = load_separators(data, thisAsset, separators_directory, from_txt=1)
nE = separators.index[0+1]-separators.index[0]+1
# check if number of events is not enough to build two features and one return
if nE>=2*data.nEventsPerStat:
    print("\t"+" s {0:d} of {1:d}".format(int(0/2),int(len(separators)/2-1))+
          ". From "+separators.DateTime.iloc[0]+" to "+separators.DateTime.iloc[0+1])
    DateTime = group_raw["DateTime"]
    SymbolBid = group_raw["SymbolBid"]
    SymbolAsk = group_raw["SymbolAsk"]
    print(DateTime.shape)
    #features = get_features_from_tsfresh(data, DateTime, SymbolBid)

0. AUDCAD
	 s 0 of 21. From 2016.01.04 00:00:08 to 2016.10.13 15:41:59
(83840918,)


In [1]:
# Snippet for tsfresh feature extraction
import numpy as np
import pandas as pd
from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
import datetime as dt

nExS = data.nEventsPerStat
mW = data.movingWindow
secsInDay = 86400.0
# return per bid
#returnBid = SymbolBid.iloc[nExS:]-SymbolBid.iloc[:-nExS+1]
nE = SymbolBid.shape[0]
# max number of features
m = int(np.floor((nE/nExS-1)*nExS/mW)+1)
# format TSFRESH input
features = pd.DataFrame()
list_inputs = []
list_DateTime = []
batch_size = 100
par_batches = 10#int(np.ceil(m/batch_size))
l_index = 0
# loop over batched
for b in range(par_batches):
    # get m of batch
    m_i = np.min([batch_size, m-b*batch_size])
    i_range = b*batch_size*nExS
    e_range = (b*batch_size+m_i)*nExS
    #print(range(i_range,e_range))
    print("Batch "+str(b)+"out of "+str(par_batches-1))
    input_ts = pd.DataFrame(data=0.0,index=range(i_range,e_range),columns=['SymbolBid','id','time'])
    datetimes = pd.DataFrame(data='',index=range(i_range,e_range),columns=['DateTime'])
    # loop over this batch
    for mm in range(m_i):
        # get indexes
        startIndex = l_index+mm*mW
        endIndex = startIndex+nExS
        # period range
        thisPeriod = range(startIndex,endIndex)
        input_ts.SymbolBid.iloc[mm*nExS:(mm+1)*nExS] = SymbolBid[thisPeriod]
        datetimes.DateTime.iloc[mm*nExS:(mm+1)*nExS] = DateTime[thisPeriod]
        input_ts.id.iloc[mm*nExS:(mm+1)*nExS] = b*batch_size+mm
        input_ts.time.iloc[mm*nExS:(mm+1)*nExS] = range(nExS)
    l_index = startIndex+mW
    #print(input_ts.shape)
    #print(input_ts.head())
    #print(input_ts.iloc[-1])
    #list_inputs.append(input_ts)
    #list_DateTime.append(datetimes)
    # build feature vector
    this_feats = extract_features(input_ts, column_id="id", column_sort="time")
    features = features.append(this_feats)
    print(features.shape)


NameError: name 'data' is not defined

In [24]:
# Snippet for tsfresh feature extraction AND selection

import numpy as np
import pandas as pd
from tsfresh import extract_features

# extract featues
nExS = data.nEventsPerStat
mW = data.movingWindow
secsInDay = 86400.0
# return per bid
#returnBid = SymbolBid.iloc[nExS:]-SymbolBid.iloc[:-nExS+1]
nE = SymbolBid.shape[0]
# max number of features
m = int(np.floor((nE/nExS-1)*nExS/mW)+1)
# format TSFRESH input
features = pd.DataFrame()
list_inputs = []
list_DateTime = []
batch_size = 100
par_batches = 100#int(np.ceil(m/batch_size))
l_index = 0
# loop over batched
for b in range(par_batches):
    # get m of batch
    m_i = np.min([batch_size, m-b*batch_size])
    i_range = b*batch_size*nExS
    e_range = (b*batch_size+m_i)*nExS
    #print(range(i_range,e_range))
    print("Batch "+str(b)+" out of "+str(par_batches-1))
    input_ts = pd.DataFrame(data=0.0,index=range(i_range,e_range),columns=['SymbolBid','id','time'])
    datetimes = pd.DataFrame(data='',index=range(i_range,e_range),columns=['DateTime'])
    # loop over this batch
    for mm in range(m_i):
        # get indexes
        startIndex = l_index+mm*mW
        endIndex = startIndex+nExS
        # period range
        thisPeriod = range(startIndex,endIndex)
        input_ts.SymbolBid.iloc[mm*nExS:(mm+1)*nExS] = SymbolBid[thisPeriod]
        datetimes.DateTime.iloc[mm*nExS:(mm+1)*nExS] = DateTime[thisPeriod]
        input_ts.id.iloc[mm*nExS:(mm+1)*nExS] = b*batch_size+mm
        input_ts.time.iloc[mm*nExS:(mm+1)*nExS] = range(nExS)
    l_index = startIndex+mW
    #print(input_ts.shape)
    #print(input_ts.head())
    #print(input_ts.iloc[-1])
    #list_inputs.append(input_ts)
    #list_DateTime.append(datetimes)
    # build feature vector
    this_feats = extract_features(input_ts, column_id="id", column_sort="time")
    features = features.append(this_feats)
    print(features.shape)

# filter features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
import datetime as dt
# get outputs
init_date = dt.datetime.strftime(dt.datetime.strptime(
            separators.DateTime.iloc[0],'%Y.%m.%d %H:%M:%S'),'%y%m%d%H%M%S')
end_date = dt.datetime.strftime(dt.datetime.strptime(
            separators.DateTime.iloc[0+1],'%Y.%m.%d %H:%M:%S'),'%y%m%d%H%M%S')
group_name = thisAsset+'/'+init_date+end_date
filename_prep_IO = (hdf5_directory+'IO_mW'+str(data.movingWindow)+'_nE'+
                        str(data.nEventsPerStat)+'_nF'+str(data.nFeatures)+'.hdf5')
f_prep_IO = h5py.File(filename_prep_IO,'r')
group = f_prep_IO[group_name]
returns = group['returns']
y = returns[:features.shape[0],data.lookAheadIndex]
# filter features
impute(features)
features_filtered = select_features(features, y)
print(features_filtered.head())
print(features_filtered.shape)

Batch 0 out of 99



Feature Extraction: 100%|██████████████████████████████████████████████████████████████████████████| 10/10 [04:09<00:00, 17.35s/it]


(100, 794)
Batch 1 out of 99



Feature Extraction: 100%|██████████████████████████████████████████████████████████████████████████| 10/10 [02:19<00:00, 12.16s/it]


(200, 794)
Batch 2 out of 99


...

(7900, 794)
Batch 79 out of 99



Feature Extraction: 100%|██████████████████████████████████████████████████████████████████████████| 10/10 [01:53<00:00,  9.82s/it]


(8000, 794)
Batch 80 out of 99



Feature Extraction:   0%|                                                                                   | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [25]:
# Snippet for tsfresh feature selection

from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
import datetime as dt
# get outputs
init_date = dt.datetime.strftime(dt.datetime.strptime(
            separators.DateTime.iloc[0],'%Y.%m.%d %H:%M:%S'),'%y%m%d%H%M%S')
end_date = dt.datetime.strftime(dt.datetime.strptime(
            separators.DateTime.iloc[0+1],'%Y.%m.%d %H:%M:%S'),'%y%m%d%H%M%S')
group_name = thisAsset+'/'+init_date+end_date
filename_prep_IO = (hdf5_directory+'IO_mW'+str(data.movingWindow)+'_nE'+
                        str(data.nEventsPerStat)+'_nF'+str(data.nFeatures)+'.hdf5')
f_prep_IO = h5py.File(filename_prep_IO,'r')
group = f_prep_IO[group_name]
returns = group['returns']
y = returns[:features.shape[0],data.lookAheadIndex]
# filter features
impute(features)
features_filtered = select_features(features, y)
print(features_filtered.head())
print(features_filtered.shape)

variable  SymbolBid__maximum  SymbolBid__minimum  SymbolBid__quantile__q_0.9  \
id                                                                             
0.0                  1.01031             1.00595                    1.008620   
1.0                  1.01031             1.00671                    1.009950   
2.0                  1.01031             1.00671                    1.010030   
3.0                  1.01153             1.00671                    1.010150   
4.0                  1.01169             1.00714                    1.011111   

variable  SymbolBid__quantile__q_0.1  \
id                                     
0.0                         1.006840   
1.0                         1.006939   
2.0                         1.007199   
3.0                         1.007220   
4.0                         1.007480   

variable  SymbolBid__fft_coefficient__coeff_70__attr_"imag"  \
id                                                            
0.0                             

In [18]:
#print(input_ts.iloc[200:210])
#print(input_ts.iloc[1100:1110])
#print(input_ts.iloc[2000:2010])
import datetime as dt
init_date = dt.datetime.strftime(dt.datetime.strptime(
            separators.DateTime.iloc[0],'%Y.%m.%d %H:%M:%S'),'%y%m%d%H%M%S')
end_date = dt.datetime.strftime(dt.datetime.strptime(
            separators.DateTime.iloc[0+1],'%Y.%m.%d %H:%M:%S'),'%y%m%d%H%M%S')
group_name = thisAsset+'/'+init_date+end_date
# get group from filef_prep_IO
filename_prep_IO = (hdf5_directory+'IO_mW'+str(data.movingWindow)+'_nE'+
                        str(data.nEventsPerStat)+'_nF'+str(data.nFeatures)+'.hdf5')
f_prep_IO = h5py.File(filename_prep_IO,'r')
group = f_prep_IO[group_name]
returns = group['returns']
print(returns.shape)
y = returns[:features.shape[0],data.lookAheadIndex]
print(y.shape)

(309819, 7)
(1000,)


In [26]:
features_filtered.to_csv('./feats_filt.txt')

In [21]:
print(features_filtered.head())
print(features_filtered.shape)

variable  SymbolBid__change_quantiles__f_agg_"var"__isabs_False__qh_0.2__ql_0.0  \
id                                                                                
0.0                                            2.507823e-09                       
1.0                                            2.500400e-09                       
2.0                                            3.540164e-09                       
3.0                                            3.611154e-09                       
4.0                                            1.311085e-09                       

variable  SymbolBid__change_quantiles__f_agg_"var"__isabs_True__qh_0.2__ql_0.0  \
id                                                                               
0.0                                            1.639956e-09                      
1.0                                            1.648129e-09                      
2.0                                            1.931948e-09                      
3.0     

[5 rows x 36 columns]
(1000, 36)


In [ ]:
# run get_features_from_tsfresh for all assets and chuncks

import numpy as np
import pandas as pd
from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
import datetime as dt

list_features_filtered = []
for ass in data.assets:
    thisAsset = data.AllAssets[str(ass)]
    print(str(ass)+". "+thisAsset)
    #tic = time.time()
    # open file for read
    
    group_raw = f_raw[thisAsset]
    #bid_means[ass_idx] = np.mean(group_raw["SymbolBid"])
    # load separators
    separators = load_separators(data, thisAsset, separators_directory, from_txt=1)
    # loop over separators
    for s in range(0,len(separators)-1,2):
        nE = separators.index[s+1]-separators.index[s]+1
        # check if number of events is not enough to build two features and one return
        if nE>=2*data.nEventsPerStat:
            print("\t"+" s {0:d} of {1:d}".format(int(s/2),int(len(separators)/2-1))+
                      ". From "+separators.DateTime.iloc[s]+" to "+separators.DateTime.iloc[s+1])
            DateTime = group_raw["DateTime"]
            SymbolBid = group_raw["SymbolBid"]
            SymbolAsk = group_raw["SymbolAsk"]
            features_filtered = get_features_from_tsfresh(data, DateTime, SymbolBid)
            features_filtered.to_csv('../features/'+thisAsset+'c'+str(int(s/2)))
            list_features_filtered.append(features_filtered)
        else:
            print("\ts {0:d} of {1:d}. Not enough entries. Skipped.".format(int(s/2),int(len(separators)/2-1)))

In [8]:
# Wrapper over assets and chuncks for feature extraction AND selection

import h5py
import numpy as np
import pandas as pd
from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
import datetime as dt
from inputs import Data, load_separators

data = Data(movingWindow=100,nEventsPerStat=1000)

# load raw data
hdf5_directory = 'D:/SDC/py/HDF5/'
filename_raw = hdf5_directory+'tradeinfo.hdf5'
separators_directory = hdf5_directory+'separators/'
f_raw = h5py.File(filename_raw,'r')

filename_prep_IO = (hdf5_directory+'IO_mW'+str(data.movingWindow)+'_nE'+
                    str(data.nEventsPerStat)+'_nF'+str(data.nFeatures)+'.hdf5')
f_prep_IO = h5py.File(filename_prep_IO,'r')

list_features_filtered = []

for ass in data.assets[:1]:
    thisAsset = data.AllAssets[str(ass)]
    print(str(ass)+". "+thisAsset)
    #tic = time.time()
    # open file for read
    
    group_raw = f_raw[thisAsset]
    DateTime = group_raw["DateTime"]
    SymbolBid = group_raw["SymbolBid"]
    SymbolAsk = group_raw["SymbolAsk"]
    #bid_means[ass_idx] = np.mean(group_raw["SymbolBid"])
    # load separators
    separators = load_separators(data, thisAsset, separators_directory, from_txt=1)
    # loop over separators
    for s in range(0,1,2):#len(separators)-1
        nE = separators.index[s+1]-separators.index[s]+1
        # check if number of events is not enough to build two features and one return
        if nE>=2*data.nEventsPerStat:
            print("\t"+" s {0:d} of {1:d}".format(int(s/2),int(len(separators)/2-1))+
                      ". From "+separators.DateTime.iloc[s]+" to "+separators.DateTime.iloc[s+1])
            
            # get outputs
            init_date = dt.datetime.strftime(dt.datetime.strptime(
                        separators.DateTime.iloc[s],'%Y.%m.%d %H:%M:%S'),'%y%m%d%H%M%S')
            end_date = dt.datetime.strftime(dt.datetime.strptime(
                        separators.DateTime.iloc[s+1],'%Y.%m.%d %H:%M:%S'),'%y%m%d%H%M%S')
            group_name = thisAsset+'/'+init_date+end_date
            
            group = f_prep_IO[group_name]
            returns = group['returns']
            
            SB = SymbolBid[separators.index[s]:separators.index[s+1]+1]
            DT = DateTime[separators.index[s]:separators.index[s+1]+1]
            
            # extract featues
            nExS = data.nEventsPerStat
            mW = data.movingWindow
            secsInDay = 86400.0
            # return per bid
            #returnBid = SymbolBid.iloc[nExS:]-SymbolBid.iloc[:-nExS+1]
            nE = SB.shape[0]
            # max number of features
            m = int(np.floor((nE/nExS-1)*nExS/mW)+1)
            # format TSFRESH input
            features = pd.DataFrame()
            list_inputs = []
            list_DateTime = []
            batch_size = 1
            par_batches = min(int(np.ceil(m/batch_size)),2)
            l_index = 0
            # loop over batched
            for b in range(par_batches):
                # get m of batch
                m_i = np.min([batch_size, m-b*batch_size])
                i_range = b*batch_size*nExS
                e_range = (b*batch_size+m_i)*nExS
                #print(range(i_range,e_range))
                print("Batch "+str(b)+" out of "+str(par_batches-1))
                input_ts = pd.DataFrame(data=0.0,index=range(i_range,e_range),columns=['SymbolBid','id','time'])
                datetimes = pd.DataFrame(data='',index=range(i_range,e_range),columns=['DateTime'])
                # loop over this batch
                for mm in range(m_i):
                    # get indexes
                    startIndex = l_index+mm*mW
                    endIndex = startIndex+nExS
                    # period range
                    thisPeriod = range(startIndex,endIndex)
                    input_ts.SymbolBid.iloc[mm*nExS:(mm+1)*nExS] = SB[thisPeriod]
                    datetimes.DateTime.iloc[mm*nExS:(mm+1)*nExS] = DT[thisPeriod]
                    input_ts.id.iloc[mm*nExS:(mm+1)*nExS] = b*batch_size+mm
                    input_ts.time.iloc[mm*nExS:(mm+1)*nExS] = range(nExS)
                l_index = startIndex+mW
                #print(input_ts.shape)
                #print(input_ts.head())
                #print(input_ts.iloc[-1])
                #list_inputs.append(input_ts)
                #list_DateTime.append(datetimes)
                # build feature vector
                this_feats = extract_features(input_ts, column_id="id", column_sort="time")
                print(this_feats.shape)
                features = features.append(this_feats)
                
            # filter features
            if returns.shape[0]<features.shape[0]:
                features = features[:returns.shape[0]-1]
            y = returns[:features.shape[0],data.lookAheadIndex]

            # filter features
            impute(features)
            features_filtered = select_features(features, y)

            print(features_filtered.shape)
            #features_filtered.iloc[:1].to_csv('../features/'+thisAsset+'c'+str(int(s/2))+'.csv')
            list_features_filtered.append(features_filtered)
        else:
            print("\ts {0:d} of {1:d}. Not enough entries. Skipped.".format(int(s/2),int(len(separators)/2-1)))

1. AUDCAD
	 s 0 of 21. From 2016.01.04 00:00:08 to 2016.10.13 15:41:59
Batch 0 out of 1


Feature Extraction: 100%|████████████████████████| 1/1 [00:03<00:00,  3.31s/it]


(1, 794)
Batch 1 out of 1


Feature Extraction: 100%|████████████████████████| 1/1 [00:03<00:00,  3.42s/it]


(1, 794)


(2, 0)


In [ ]:
# Main script: get selected features from tsfresh tool
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from inputs import Data, load_separators
import h5py
import datetime as dt
import time

def feval(funcName, *args):
    return eval(funcName)(*args)

#def get_features_tsfresh():
if 1:
    """
    Extract and save most common features based on TSFRESH tool
    """
    # config stuff
    hdf5_directory = 'D:/SDC/py/HDF5/'
    save_stats = False
    # init stuff
    filename_raw = hdf5_directory+'tradeinfo.hdf5'
    separators_directory = hdf5_directory+'separators/'
    
    f_raw = h5py.File(filename_raw,'r')
    dateTest = ([                                                   '2018.03.09',
                '2018.03.12','2018.03.13','2018.03.14','2018.03.15','2018.03.16',
                '2018.03.19','2018.03.20','2018.03.21','2018.03.22','2018.03.23',
                '2018.03.26','2018.03.27','2018.03.28','2018.03.29','2018.03.30',
                '2018.04.02','2018.04.03','2018.04.04','2018.04.05','2018.04.06',
                '2018.04.09','2018.04.10','2018.04.11','2018.04.12','2018.04.13',
                '2018.04.16','2018.04.17','2018.04.18','2018.04.19','2018.04.20',
                '2018.04.23','2018.04.24','2018.04.25','2018.04.26','2018.04.27',
                '2018.04.30','2018.05.01','2018.05.02','2018.05.03','2018.05.04',
                '2018.05.07','2018.05.08','2018.05.09','2018.05.10','2018.05.11',
                '2018.05.14','2018.05.15','2018.05.16','2018.05.17','2018.05.18',
                '2018.05.21','2018.05.22','2018.05.23','2018.05.24','2018.05.25',
                '2018.05.28','2018.05.29','2018.05.30','2018.05.31','2018.06.01',
                '2018.06.04','2018.06.05','2018.06.06','2018.06.07','2018.06.08',
                '2018.06.11','2018.06.12','2018.06.13','2018.06.14','2018.06.15',
                '2018.06.18','2018.06.19','2018.06.20','2018.06.21','2018.06.22',
                '2018.06.25','2018.06.26','2018.06.27','2018.06.28','2018.06.29',
                '2018.07.02','2018.07.03','2018.07.04','2018.07.05','2018.07.06',
                '2018.07.09','2018.07.10','2018.07.11','2018.07.12','2018.07.13',
                '2018.07.30','2018.07.31','2018.08.01','2018.08.02','2018.08.03',
                '2018.08.06','2018.08.07','2018.08.08','2018.08.09','2018.08.10']+
               ['2018.08.13','2018.08.14','2018.08.15','2018.08.16','2018.08.17',
                '2018.08.20','2018.08.21','2018.08.22','2018.08.23','2018.08.24',
                '2018.08.27','2018.08.28','2018.08.29','2018.08.30','2018.08.31',
                '2018.09.03','2018.09.04','2018.09.05','2018.09.06','2018.09.07',
                '2018.09.10','2018.09.11','2018.09.12','2018.09.13','2018.09.14',
                '2018.09.17','2018.09.18','2018.09.19','2018.09.20','2018.09.21',
                '2018.09.24','2018.09.25','2018.09.26','2018.09.27']+['2018.09.28',
                '2018.10.01','2018.10.02','2018.10.03','2018.10.04','2018.10.05',
                '2018.10.08','2018.10.09','2018.10.10','2018.10.11','2018.10.12',
                '2018.10.15','2018.10.16','2018.10.17','2018.10.18','2018.10.19',
                '2018.10.22','2018.10.23','2018.10.24','2018.10.25','2018.10.26',
                '2018.10.29','2018.10.30','2018.10.31','2018.11.01','2018.11.02',
                '2018.11.05','2018.11.06','2018.11.07','2018.11.08','2018.11.09'])
    data=Data(movingWindow=100,
              nEventsPerStat=1000,
              dateTest=dateTest,
              feature_keys_tsfresh=[i for i in range(37,68)])
    filename_features_tsf = (hdf5_directory+'feats_tsf_mW'+str(data.movingWindow)+'_nE'+
                            str(data.nEventsPerStat)+'_2.hdf5')
    batch_size = 50000
    window_size = data.nEventsPerStat
    sprite_length = data.movingWindow
    nMaxChannels = int(window_size/sprite_length)
    features_tsfresh = data.feature_keys_tsfresh
    n_feats_tsfresh = data.n_feats_tsfresh
    file_features_tsf = h5py.File(filename_features_tsf,'a')
    tic_t = time.time()
    # run over assets
    for ass in data.assets:
        tic_ass = time.time()
        # retrieve this asset's name
        thisAsset = data.AllAssets[str(ass)]
        print(str(ass)+". "+thisAsset)
        # load separators
        separators = load_separators(data, thisAsset, 
                                     separators_directory, 
                                     from_txt=1)
        # get raw data
        SymbolBid = f_raw[thisAsset]["SymbolBid"]
        # init normalization stats
        stats = {"means_t_in":np.zeros((nMaxChannels,n_feats_tsfresh)),
                "stds_t_in":np.zeros((nMaxChannels,n_feats_tsfresh)),
                "m_t_in":0}
        # run over separators
        for s in range(0,len(separators),2):#
            tic = time.time()
            print("\t chuck {0:d} of {1:d}".format(int(s/2),int(len(separators)/2-1))+
                      ". From "+separators.DateTime.iloc[s]+" to "+separators.DateTime.iloc[s+1])
            
            # number of events in this chunck
            n_events = separators.index[s+1]-separators.index[s]+1
            if n_events>=2*window_size:
                print("\t Getting features from raw data...")
                tic = time.time()
                tic_chunck = tic
                # init and end dates in string format
                init_date = dt.datetime.strftime(dt.datetime.strptime(
                        separators.DateTime.iloc[s],'%Y.%m.%d %H:%M:%S'),'%y%m%d%H%M%S')
                end_date = dt.datetime.strftime(dt.datetime.strptime(
                        separators.DateTime.iloc[s+1],'%Y.%m.%d %H:%M:%S'),'%y%m%d%H%M%S')
                # hdf5 group name
                group_name_chunck = thisAsset+'/'+init_date+end_date
                # create or retrieve group
                # create new gruop if not yet in file
                # bids in this chunck
                events = SymbolBid[separators.index[s]:separators.index[s+1]+1]
                # number of samples
                m = int(np.floor((n_events/window_size-1)*window_size/sprite_length)+1)
                # group ts in m chunks
                # number of batches
                batches = int(np.ceil(m/batch_size))
                l_index = 0
                m_counter = 0
                features = np.zeros((m,n_feats_tsfresh))
                # loop over batched
                for b in range(batches):
                    print("\t batch "+str(b)+" out of "+str(batches-1))
                    # get batch size
                    m_i = np.min([batch_size, m-b*batch_size])
                    #print("Batch "+str(b)+" out of "+str(batches-1)+", m_i="+str(m_i))
                    # init and end of event index
                    i_event = l_index
                    e_event = i_event+(m_i-1)*sprite_length+window_size
                    # init and position of the batch
                    i_batch = b*batch_size*window_size
                    e_batch = (b*batch_size+m_i)*window_size
                    # serial input
                    x_ser = events[i_event:e_event]
                    #print("x_ser length="+str(x_ser.shape[0]))
                    # loop over this batch samples
                    for mm in range(m_i):
                        # get sample indexes
                        i_sample = mm*sprite_length
                        e_sample = i_sample+window_size
                        #print("i_sample="+str(i_sample)+" e_sample="+str(e_sample))
                        x = x_ser[i_sample:e_sample]
                        c = 0
                        # extract features
                        for f in features_tsfresh:
                            group_name_feat = group_name_chunck+'/'+str(f)
                            params = data.AllFeatures[str(f)]
                            n_new_feats = params[-1]
                            if group_name_feat not in file_features_tsf:
                                params = data.AllFeatures[str(f)]
                                new_feats = feval(params[0],x,params[1:])
                                n_new_feats = params[-1]
                                features[m_counter,c:c+n_new_feats] = new_feats                                
                                #if b==0 and mm==0:
                                #    print("\t getting features of "+group_name_feat)
                            elif b==0 and mm<10:
                                print("\t "+group_name_feat+" found in file. Skipping")
                            c += n_new_feats
                        #assert(c==n_feats_tsfresh)
                        m_counter += 1
                    # serial loop
                    l_index = e_event-window_size+sprite_length
                # end of for b in range(batches):
                print("\t time for feature extraction: "+str(time.time()-tic))
                #print("\r"+"e_event="+str(e_event)+" events="+str(events.shape[0]))#, sep=' ', end='\n', flush=True
                
                c = 0
                # save features in HDF5 file
                for f in features_tsfresh:
                    group_name_feat = group_name_chunck+'/'+str(f)
                    params = data.AllFeatures[str(f)]
                    n_new_feats = params[-1]
                    #if group_name_feat in file_features_tsf:
                    #    del file_features_tsf[group_name_feat]
                    if group_name_feat not in file_features_tsf:
                        group_chunck = file_features_tsf.create_group(group_name_feat)
                        group_features = group_chunck.create_dataset("feature", (m,n_new_feats),dtype=float)
                        group_features[:,:] = features[:,c:c+n_new_feats]
                        file_features_tsf.flush()
                        print(group_name_feat+" saved in file")
                    else: # load features from HDF5 file if they are saved already
                        success = False
                        tries = 0
                        while not success and tries<10:
                            try:
                                group_chunck = file_features_tsf[group_name_feat]
                                #print("features from "+group_name_feat+" loaded")
                                success = True
                            except KeyError:
                                print("KeyError reading "+group_name_feat)
                                tries += 1
                        if not success:
                            raise KeyError(group_name_feat+" File corrupted. Delete and retrieve again")
                            
                        features[:,c:c+n_new_feats] = group_chunck["feature"]
                    c += n_new_feats
                # end of for f in features_tsfresh:
                # Calculate normalization stats
                tic = time.time()
                # open temporal file for variations
                try:
                    # create file
                    ft = h5py.File(hdf5_directory+'temp.hdf5','w')
                    # create group
                    group_temp = ft.create_group('temp')
                    # reserve memory space for variations and normalized variations
                    variations = group_temp.create_dataset("variations", (features.shape[0],features.shape[1],nMaxChannels), 
                                                           dtype=float)
                    print("\t getting variations")
                    # loop over channels
                    for r in range(nMaxChannels):
                        variations[r+1:,:,r] = features[:-(r+1),:]
                    print("\t time for variations: "+str(time.time()-tic))
                    # init stats    
                    means_in = np.zeros((nMaxChannels,features.shape[1]))
                    stds_in = np.zeros((nMaxChannels,features.shape[1]))
                    print("\t getting means and stds")
                    # loop over channels
                    for r in range(nMaxChannels):
                        #nonZeros = variations[:,0,r]!=999
                        #print(np.sum(nonZeros))
                        means_in[r,:] = np.mean(variations[nMaxChannels:,:,r],axis=0,keepdims=1)
                        stds_in[r,:] = np.std(variations[nMaxChannels:,:,r],axis=0,keepdims=1)  
                    print("\t time for stats: "+str(time.time()-tic))
                except KeyboardInterrupt:
                    ft.close()
                    print("KeyboardInterrupt! Closing file and exiting.")
                    raise KeyboardInterrupt
                ft.close()
                # end of normalization stats
                # save normalization stats
                for f in features_tsfresh:
                    group_name_feat = group_name_chunck+'/'+str(f)
                    n_new_feats = data.AllFeatures[str(f)][-1]
                    group_feature = file_features_tsf[group_name_feat]
                    # save means and variances as atributes
                    group_feature.attrs.create("means_in", means_in[:,c:c+n_new_feats], dtype=float)
                    group_feature.attrs.create("stds_in", stds_in[:,c:c+n_new_feats], dtype=float)
                    c += n_new_feats
                # update total stats
                stats["means_t_in"] += m*means_in
                stats["stds_t_in"] += m*stds_in
                stats["m_t_in"] += m
                means_in
                print("\t this chunck time: "+str(np.floor(time.time()-tic_chunck))+"s")
            else:
                print("\t chunck {0:d} of {1:d}. Not enough entries. Skipped.".format(int(s/2),int(len(separators)/2-1)))
            # end of if n_events>=2*window_size:
        # end of for s in range(0,len(separators)-1,2):
        stats["means_t_in"] = stats["means_t_in"]/stats["m_t_in"]
        stats["stds_t_in"] = stats["stds_t_in"]/stats["m_t_in"]
        means_t_in = stats["means_t_in"]
        stds_t_in = stats["stds_t_in"]
        if save_stats:
            c = 0
            for f in features_tsfresh:
                group_name_feat = thisAsset
                n_new_feats = data.AllFeatures[str(f)][-1]
                group_feature = file_features_tsf[group_name_feat]
                # save means and variances as atributes
                group_feature.attrs.create("means_in"+str(f), means_t_in[:,c:c+n_new_feats], dtype=float)
                group_feature.attrs.create("stds_in"+str(f), stds_t_in[:,c:c+n_new_feats], dtype=float)
                c += n_new_feats
        print("\t this asset time: "+str(np.floor(time.time()-tic_ass)/60)+" minutes")
    # end of for ass in data.assets:
    file_features_tsf.close()
    print("DONE. Total time: "+str(np.floor(time.time()-tic_t)/60)+" minutes")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1. AUDCAD
	 chuck 0 of 21. From 2016.01.04 00:00:08 to 2016.10.13 15:41:59
	 Getting features from raw data...
	 batch 0 out of 6
	 batch 1 out of 6
	 batch 2 out of 6
	 batch 3 out of 6
	 batch 4 out of 6
	 batch 5 out of 6
	 batch 6 out of 6
	 time for feature extraction: 4282.6673991680145
AUDCAD/160104000008161013154159/37 saved in file
AUDCAD/160104000008161013154159/38 saved in file
AUDCAD/160104000008161013154159/39 saved in file
AUDCAD/160104000008161013154159/40 saved in file
AUDCAD/160104000008161013154159/41 saved in file
AUDCAD/160104000008161013154159/42 saved in file
AUDCAD/160104000008161013154159/43 saved in file
AUDCAD/160104000008161013154159/44 saved in file
AUDCAD/160104000008161013154159/45 saved in file
AUDCAD/160104000008161013154159/46 saved in file
AUDCAD/160104000008161013154159/47 saved in file
AUDCAD/160104000008161013154159/48 saved in file
AUDCAD/16010400000816101315415

AUDCAD/170130000500170905235959/50 saved in file
AUDCAD/170130000500170905235959/51 saved in file
AUDCAD/170130000500170905235959/52 saved in file
AUDCAD/170130000500170905235959/53 saved in file
AUDCAD/170130000500170905235959/54 saved in file
AUDCAD/170130000500170905235959/55 saved in file
AUDCAD/170130000500170905235959/56 saved in file
AUDCAD/170130000500170905235959/57 saved in file
AUDCAD/170130000500170905235959/58 saved in file
AUDCAD/170130000500170905235959/59 saved in file
AUDCAD/170130000500170905235959/60 saved in file
AUDCAD/170130000500170905235959/61 saved in file
AUDCAD/170130000500170905235959/62 saved in file
AUDCAD/170130000500170905235959/63 saved in file
AUDCAD/170130000500170905235959/64 saved in file
AUDCAD/170130000500170905235959/65 saved in file
AUDCAD/170130000500170905235959/66 saved in file
AUDCAD/170130000500170905235959/67 saved in file
	 getting variations
	 time for variations: 219.0362937450409
	 getting means and stds
	 time for stats: 229.824532747

	 time for variations: 0.7535045146942139
	 getting means and stds
	 time for stats: 0.7847492694854736
	 this chunck time: 12.0s
	 chuck 9 of 21. From 2017.12.12 09:55:53 to 2017.12.22 23:58:59
	 Getting features from raw data...
	 batch 0 out of 0
	 time for feature extraction: 117.66543459892273
AUDCAD/171212095553171222235859/37 saved in file
AUDCAD/171212095553171222235859/38 saved in file
AUDCAD/171212095553171222235859/39 saved in file
AUDCAD/171212095553171222235859/40 saved in file
AUDCAD/171212095553171222235859/41 saved in file
AUDCAD/171212095553171222235859/42 saved in file
AUDCAD/171212095553171222235859/43 saved in file
AUDCAD/171212095553171222235859/44 saved in file
AUDCAD/171212095553171222235859/45 saved in file
AUDCAD/171212095553171222235859/46 saved in file
AUDCAD/171212095553171222235859/47 saved in file
AUDCAD/171212095553171222235859/48 saved in file
AUDCAD/171212095553171222235859/49 saved in file
AUDCAD/171212095553171222235859/50 saved in file
AUDCAD/1712120

AUDCAD/180125085348180308235859/50 saved in file
AUDCAD/180125085348180308235859/51 saved in file
AUDCAD/180125085348180308235859/52 saved in file
AUDCAD/180125085348180308235859/53 saved in file
AUDCAD/180125085348180308235859/54 saved in file
AUDCAD/180125085348180308235859/55 saved in file
AUDCAD/180125085348180308235859/56 saved in file
AUDCAD/180125085348180308235859/57 saved in file
AUDCAD/180125085348180308235859/58 saved in file
AUDCAD/180125085348180308235859/59 saved in file
AUDCAD/180125085348180308235859/60 saved in file
AUDCAD/180125085348180308235859/61 saved in file
AUDCAD/180125085348180308235859/62 saved in file
AUDCAD/180125085348180308235859/63 saved in file
AUDCAD/180125085348180308235859/64 saved in file
AUDCAD/180125085348180308235859/65 saved in file
AUDCAD/180125085348180308235859/66 saved in file
AUDCAD/180125085348180308235859/67 saved in file
	 getting variations
	 time for variations: 50.813018798828125
	 getting means and stds
	 time for stats: 53.402623176

	 time for variations: 41.314481019973755
	 getting means and stds
	 time for stats: 44.15477180480957
	 this chunck time: 636.0s
	 chuck 18 of 21. From 2018.07.30 00:05:00 to 2018.08.10 23:58:59
	 Getting features from raw data...
	 batch 0 out of 0
	 time for feature extraction: 117.66196513175964
AUDCAD/180730000500180810235859/37 saved in file
AUDCAD/180730000500180810235859/38 saved in file
AUDCAD/180730000500180810235859/39 saved in file
AUDCAD/180730000500180810235859/40 saved in file
AUDCAD/180730000500180810235859/41 saved in file
AUDCAD/180730000500180810235859/42 saved in file
AUDCAD/180730000500180810235859/43 saved in file
AUDCAD/180730000500180810235859/44 saved in file
AUDCAD/180730000500180810235859/45 saved in file
AUDCAD/180730000500180810235859/46 saved in file
AUDCAD/180730000500180810235859/47 saved in file
AUDCAD/180730000500180810235859/48 saved in file
AUDCAD/180730000500180810235859/49 saved in file
AUDCAD/180730000500180810235859/50 saved in file
AUDCAD/180730

...

EURNZD/180102030000180110211301/57 saved in file
EURNZD/180102030000180110211301/58 saved in file
EURNZD/180102030000180110211301/59 saved in file
EURNZD/180102030000180110211301/60 saved in file
EURNZD/180102030000180110211301/61 saved in file
EURNZD/180102030000180110211301/62 saved in file
EURNZD/180102030000180110211301/63 saved in file
EURNZD/180102030000180110211301/64 saved in file
EURNZD/180102030000180110211301/65 saved in file
EURNZD/180102030000180110211301/66 saved in file
EURNZD/180102030000180110211301/67 saved in file
	 getting variations
	 time for variations: 10.118964195251465
	 getting means and stds
	 time for stats: 10.641563415527344
	 this chunck time: 90.0s
	 chuck 20 of 29. From 2018.01.11 14:39:29 to 2018.01.25 08:17:38
	 Getting features from raw data...
	 batch 0 out of 0
	 time for feature extraction: 152.38911390304565
EURNZD/180111143929180125081738/37 saved in file
EURNZD/180111143929180125081738/38 saved in file
EURNZD/180111143929180125081738/39 saved 

EURNZD/180412095909180508032749/38 saved in file
EURNZD/180412095909180508032749/39 saved in file
EURNZD/180412095909180508032749/40 saved in file
EURNZD/180412095909180508032749/41 saved in file
EURNZD/180412095909180508032749/42 saved in file
EURNZD/180412095909180508032749/43 saved in file
EURNZD/180412095909180508032749/44 saved in file
EURNZD/180412095909180508032749/45 saved in file
EURNZD/180412095909180508032749/46 saved in file
EURNZD/180412095909180508032749/47 saved in file
EURNZD/180412095909180508032749/48 saved in file
EURNZD/180412095909180508032749/49 saved in file
EURNZD/180412095909180508032749/50 saved in file
EURNZD/180412095909180508032749/51 saved in file
EURNZD/180412095909180508032749/52 saved in file
EURNZD/180412095909180508032749/53 saved in file
EURNZD/180412095909180508032749/54 saved in file
EURNZD/180412095909180508032749/55 saved in file
EURNZD/180412095909180508032749/56 saved in file
EURNZD/180412095909180508032749/57 saved in file
EURNZD/1804120959091

EURNZD/180823131434180824235851/57 saved in file
EURNZD/180823131434180824235851/58 saved in file
EURNZD/180823131434180824235851/59 saved in file
EURNZD/180823131434180824235851/60 saved in file
EURNZD/180823131434180824235851/61 saved in file
EURNZD/180823131434180824235851/62 saved in file
EURNZD/180823131434180824235851/63 saved in file
EURNZD/180823131434180824235851/64 saved in file
EURNZD/180823131434180824235851/65 saved in file
EURNZD/180823131434180824235851/66 saved in file
EURNZD/180823131434180824235851/67 saved in file
	 getting variations
	 time for variations: 1.912076711654663
	 getting means and stds
	 time for stats: 2.035743236541748
	 this chunck time: 18.0s
	 chuck 29 of 29. From 2018.08.27 19:03:32 to 2018.11.09 23:59:06
	 Getting features from raw data...
	 batch 0 out of 1
	 batch 1 out of 1
	 time for feature extraction: 678.2739689350128
EURNZD/180827190332181109235906/37 saved in file
EURNZD/180827190332181109235906/38 saved in file
EURNZD/180827190332181109

	 batch 0 out of 0
	 time for feature extraction: 158.4063606262207
EURUSD/161226090059170111184220/37 saved in file
EURUSD/161226090059170111184220/38 saved in file
EURUSD/161226090059170111184220/39 saved in file
EURUSD/161226090059170111184220/40 saved in file
EURUSD/161226090059170111184220/41 saved in file
EURUSD/161226090059170111184220/42 saved in file
EURUSD/161226090059170111184220/43 saved in file
EURUSD/161226090059170111184220/44 saved in file
EURUSD/161226090059170111184220/45 saved in file
EURUSD/161226090059170111184220/46 saved in file
EURUSD/161226090059170111184220/47 saved in file
EURUSD/161226090059170111184220/48 saved in file
EURUSD/161226090059170111184220/49 saved in file
EURUSD/161226090059170111184220/50 saved in file
EURUSD/161226090059170111184220/51 saved in file
EURUSD/161226090059170111184220/52 saved in file
EURUSD/161226090059170111184220/53 saved in file
EURUSD/161226090059170111184220/54 saved in file
EURUSD/161226090059170111184220/55 saved in file
E

EURUSD/170815140741170825200623/52 saved in file
EURUSD/170815140741170825200623/53 saved in file
EURUSD/170815140741170825200623/54 saved in file
EURUSD/170815140741170825200623/55 saved in file
EURUSD/170815140741170825200623/56 saved in file
EURUSD/170815140741170825200623/57 saved in file
EURUSD/170815140741170825200623/58 saved in file
EURUSD/170815140741170825200623/59 saved in file
EURUSD/170815140741170825200623/60 saved in file
EURUSD/170815140741170825200623/61 saved in file
EURUSD/170815140741170825200623/62 saved in file
EURUSD/170815140741170825200623/63 saved in file
EURUSD/170815140741170825200623/64 saved in file
EURUSD/170815140741170825200623/65 saved in file
EURUSD/170815140741170825200623/66 saved in file
EURUSD/170815140741170825200623/67 saved in file
	 getting variations
	 time for variations: 16.747335195541382
	 getting means and stds
	 time for stats: 17.53525733947754
	 this chunck time: 158.0s
	 chuck 8 of 27. From 2017.08.28 00:05:02 to 2017.09.05 02:36:01


In [5]:
file_features_tsf.close()

In [ ]:
# Script save stats from tsfresh features 
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from inputs import Data, load_separators
import h5py
import datetime as dt
import time

def feval(funcName, *args):
    return eval(funcName)(*args)

#def get_features_tsfresh():
if 1:
    """
    Extract and save most common features based on TSFRESH tool
    """
    # config stuff
    hdf5_directory = 'D:/SDC/py/HDF5/'
    save_stats = False
    # init stuff
    filename_raw = hdf5_directory+'tradeinfo.hdf5'
    separators_directory = hdf5_directory+'separators/'
    
    f_raw = h5py.File(filename_raw,'r')
    
    data=Data(movingWindow=100,
              nEventsPerStat=1000,
              dateTest = [                                          '2018.03.09',
                '2018.03.12','2018.03.13','2018.03.14','2018.03.15','2018.03.16',
                '2018.03.19','2018.03.20','2018.03.21','2018.03.22','2018.03.23',
                '2018.03.26','2018.03.27','2018.03.28','2018.03.29','2018.03.30',
                '2018.04.02','2018.04.03','2018.04.04','2018.04.05','2018.04.06',
                '2018.04.09','2018.04.10','2018.04.11','2018.04.12','2018.04.13',
                '2018.04.16','2018.04.17','2018.04.18','2018.04.19','2018.04.20',
                '2018.04.23','2018.04.24','2018.04.25','2018.04.26','2018.04.27',
                '2018.04.30','2018.05.01','2018.05.02','2018.05.03','2018.05.04',
                '2018.05.07','2018.05.08','2018.05.09','2018.05.10','2018.05.11',
                '2018.05.14','2018.05.15','2018.05.16','2018.05.17','2018.05.18',
                '2018.05.21','2018.05.22','2018.05.23','2018.05.24','2018.05.25',
                '2018.05.28','2018.05.29','2018.05.30','2018.05.31','2018.06.01',
                '2018.06.04','2018.06.05','2018.06.06','2018.06.07','2018.06.08',
                '2018.06.11','2018.06.12','2018.06.13','2018.06.14','2018.06.15',
                '2018.06.18','2018.06.19','2018.06.20','2018.06.21','2018.06.22',
                '2018.06.25','2018.06.26','2018.06.27','2018.06.28','2018.06.29',
                '2018.07.02','2018.07.03','2018.07.04','2018.07.05','2018.07.06',
                '2018.07.09','2018.07.10','2018.07.11','2018.07.12','2018.07.13',
                '2018.07.30','2018.07.31','2018.08.01','2018.08.02','2018.08.03',
                '2018.08.06','2018.08.07','2018.08.08','2018.08.09','2018.08.10']+
               ['2018.08.13','2018.08.14','2018.08.15','2018.08.16','2018.08.17',
                '2018.08.20','2018.08.21','2018.08.22','2018.08.23','2018.08.24',
                '2018.08.27','2018.08.28','2018.08.29','2018.08.30','2018.08.31',
                '2018.09.03','2018.09.04','2018.09.05','2018.09.06','2018.09.07',
                '2018.09.10','2018.09.11','2018.09.12','2018.09.13','2018.09.14',
                '2018.09.17','2018.09.18','2018.09.19','2018.09.20','2018.09.21',
                '2018.09.24','2018.09.25','2018.09.26','2018.09.27'])
    filename_features_tsf = (hdf5_directory+'feats_tsf_mW'+str(data.movingWindow)+'_nE'+
                            str(data.nEventsPerStat)+'.hdf5')
    batch_size = 50000
    window_size = data.nEventsPerStat
    sprite_length = data.movingWindow
    nMaxChannels = int(window_size/sprite_length)
    features_tsfresh = data.feature_keys_tsfresh
    n_feats_tsfresh = data.n_feats_tsfresh
    file_features_tsf = h5py.File(filename_features_tsf,'a')
    tic_t = time.time()
    # run over assets
    for ass in data.assets:
        tic_ass = time.time()
        # retrieve this asset's name
        thisAsset = data.AllAssets[str(ass)]
        print(str(ass)+". "+thisAsset)
        # load separators
        separators = load_separators(data, thisAsset, 
                                     separators_directory, 
                                     from_txt=1)
        # get raw data
        SymbolBid = f_raw[thisAsset]["SymbolBid"]
        # init normalization stats
        stats = {"means_t_in":np.zeros((nMaxChannels,n_feats_tsfresh)),
                "stds_t_in":np.zeros((nMaxChannels,n_feats_tsfresh)),
                "m_t_in":0}
        # run over separators
        for s in range(len(separators)-2,len(separators),2):#
            tic = time.time()
            print("\t chuck {0:d} of {1:d}".format(int(s/2),int(len(separators)/2-1))+
                      ". From "+separators.DateTime.iloc[s]+" to "+separators.DateTime.iloc[s+1])
            
            # number of events in this chunck
            n_events = separators.index[s+1]-separators.index[s]+1
            if n_events>=2*window_size:
                print("\t Getting features from raw data...")
                tic = time.time()
                tic_chunck = tic
                # init and end dates in string format
                init_date = dt.datetime.strftime(dt.datetime.strptime(
                        separators.DateTime.iloc[s],'%Y.%m.%d %H:%M:%S'),'%y%m%d%H%M%S')
                end_date = dt.datetime.strftime(dt.datetime.strptime(
                        separators.DateTime.iloc[s+1],'%Y.%m.%d %H:%M:%S'),'%y%m%d%H%M%S')
                # hdf5 group name
                group_name_chunck = thisAsset+'/'+init_date+end_date
                # create or retrieve group
                # create new gruop if not yet in file
                # bids in this chunck
                m = int(np.floor((n_events/window_size-1)*window_size/sprite_length)+1)
                features = np.zeros((m,n_feats_tsfresh))
                c = 0
                # load features in HDF5 file
                for f in features_tsfresh:
                    group_name_feat = group_name_chunck+'/'+str(f)
                    params = data.AllFeatures[str(f)]
                    n_new_feats = params[-1]
                    #if group_name_feat in file_features_tsf:
                    #    del file_features_tsf[group_name_feat]
                    if group_name_feat not in file_features_tsf:
                        raise ValueError(group_name_feat+ "not in "+str(file_features_tsf))
                    else: # load features from HDF5 file if they are saved already
                        success = False
                        tries = 0
                        while not success and tries<10:
                            try:
                                group_chunck = file_features_tsf[group_name_feat]
                                #print("features from "+group_name_feat+" loaded")
                                success = True
                            except KeyError:
                                print("KeyError reading "+group_name_feat)
                                tries += 1
                        if not success:
                            raise KeyError(group_name_feat+" File corrupted. Delete and retrieve again")
                            
                        features[:,c:c+n_new_feats] = group_chunck["feature"]
                    c += n_new_feats
                # end of for f in features_tsfresh:
                # Calculate normalization stats
                tic = time.time()
                # open temporal file for variations
                try:
                    # create file
                    ft = h5py.File(hdf5_directory+'temp.hdf5','w')
                    # create group
                    group_temp = ft.create_group('temp')
                    # reserve memory space for variations and normalized variations
                    variations = group_temp.create_dataset("variations", (features.shape[0],features.shape[1],nMaxChannels), 
                                                           dtype=float)
                    print("\t getting variations")
                    # loop over channels
                    for r in range(nMaxChannels):
                        variations[r+1:,:,r] = features[:-(r+1),:]
                    print("\t time for variations: "+str(time.time()-tic))
                    # init stats    
                    means_in = np.zeros((nMaxChannels,features.shape[1]))
                    stds_in = np.zeros((nMaxChannels,features.shape[1]))
                    print("\t getting means and stds")
                    # loop over channels
                    for r in range(nMaxChannels):
                        #nonZeros = variations[:,0,r]!=999
                        #print(np.sum(nonZeros))
                        means_in[r,:] = np.mean(variations[nMaxChannels:,:,r],axis=0,keepdims=1)
                        stds_in[r,:] = np.std(variations[nMaxChannels:,:,r],axis=0,keepdims=1)  
                    print("\t time for stats: "+str(time.time()-tic))
                except KeyboardInterrupt:
                    ft.close()
                    print("KeyboardInterrupt! Closing file and exiting.")
                    raise KeyboardInterrupt
                ft.close()
                # end of normalization stats
                # save normalization stats
                for f in features_tsfresh:
                    group_name_feat = group_name_chunck+'/'+str(f)
                    n_new_feats = data.AllFeatures[str(f)][-1]
                    group_feature = file_features_tsf[group_name_feat]
                    # save means and variances as atributes
                    group_feature.attrs.create("means_in", means_in[:,c:c+n_new_feats], dtype=float)
                    group_feature.attrs.create("stds_in", stds_in[:,c:c+n_new_feats], dtype=float)
                    c += n_new_feats
                # update total stats
                stats["means_t_in"] += m*means_in
                stats["stds_t_in"] += m*stds_in
                stats["m_t_in"] += m
                means_in
                print("\t this chunck time: "+str(np.floor(time.time()-tic_chunck))+"s")
            else:
                print("\t chunck {0:d} of {1:d}. Not enough entries. Skipped.".format(int(s/2),int(len(separators)/2-1)))
            # end of if n_events>=2*window_size:
        # end of for s in range(0,len(separators)-1,2):
        means_t_in = stats["means_t_in"]/stats["m_t_in"]
        stds_t_in = stats["stds_t_in"]/stats["m_t_in"]
        if save_stats:
            c = 0
            for f in features_tsfresh:
                group_name_feat = thisAsset
                n_new_feats = data.AllFeatures[str(f)][-1]
                group_feature = file_features_tsf[group_name_feat]
                # save means and variances as atributes
                group_feature.attrs.create("means_in"+str(f), means_t_in[:,c:c+n_new_feats], dtype=float)
                group_feature.attrs.create("stds_in"+str(f), stds_t_in[:,c:c+n_new_feats], dtype=float)
                c += n_new_feats
        print("\t this asset time: "+str(np.floor(time.time()-tic_ass)/60)+" minutes")
    # end of for ass in data.assets:
    file_features_tsf.close()
    print("DONE. Total time: "+str(np.floor(time.time()-tic_t)/60)+" minutes")

In [3]:
# Sanity check of feautures file

import numpy as np
import pandas as pd
from inputs import Data, load_separators
import h5py
import datetime as dt

hdf5_directory = 'D:/SDC/py/HDF5/'
filename_raw = hdf5_directory+'tradeinfo.hdf5'
separators_directory = hdf5_directory+'separators/'
data=Data(movingWindow=100,
              nEventsPerStat=1000,
              dateTest = [                                          '2018.03.09',
                '2018.03.12','2018.03.13','2018.03.14','2018.03.15','2018.03.16',
                '2018.03.19','2018.03.20','2018.03.21','2018.03.22','2018.03.23',
                '2018.03.26','2018.03.27','2018.03.28','2018.03.29','2018.03.30',
                '2018.04.02','2018.04.03','2018.04.04','2018.04.05','2018.04.06',
                '2018.04.09','2018.04.10','2018.04.11','2018.04.12','2018.04.13',
                '2018.04.16','2018.04.17','2018.04.18','2018.04.19','2018.04.20',
                '2018.04.23','2018.04.24','2018.04.25','2018.04.26','2018.04.27',
                '2018.04.30','2018.05.01','2018.05.02','2018.05.03','2018.05.04',
                '2018.05.07','2018.05.08','2018.05.09','2018.05.10','2018.05.11',
                '2018.05.14','2018.05.15','2018.05.16','2018.05.17','2018.05.18',
                '2018.05.21','2018.05.22','2018.05.23','2018.05.24','2018.05.25',
                '2018.05.28','2018.05.29','2018.05.30','2018.05.31','2018.06.01',
                '2018.06.04','2018.06.05','2018.06.06','2018.06.07','2018.06.08',
                '2018.06.11','2018.06.12','2018.06.13','2018.06.14','2018.06.15',
                '2018.06.18','2018.06.19','2018.06.20','2018.06.21','2018.06.22',
                '2018.06.25','2018.06.26','2018.06.27','2018.06.28','2018.06.29',
                '2018.07.02','2018.07.03','2018.07.04','2018.07.05','2018.07.06',
                '2018.07.09','2018.07.10','2018.07.11','2018.07.12','2018.07.13',
                '2018.07.30','2018.07.31','2018.08.01','2018.08.02','2018.08.03',
                '2018.08.06','2018.08.07','2018.08.08','2018.08.09','2018.08.10']+
               ['2018.08.13','2018.08.14','2018.08.15','2018.08.16','2018.08.17',
                '2018.08.20','2018.08.21','2018.08.22','2018.08.23','2018.08.24',
                '2018.08.27','2018.08.28','2018.08.29','2018.08.30','2018.08.31',
                '2018.09.03','2018.09.04','2018.09.05','2018.09.06','2018.09.07',
                '2018.09.10','2018.09.11','2018.09.12','2018.09.13','2018.09.14',
                '2018.09.17','2018.09.18','2018.09.19','2018.09.20','2018.09.21',
                '2018.09.24','2018.09.25','2018.09.26','2018.09.27'],
                 feature_keys_tsfresh=[i for i in range(37,68)])
filename_features_tsf = (hdf5_directory+'feats_tsf_mW'+str(data.movingWindow)+'_nE'+
                         str(data.nEventsPerStat)+'.hdf5')
window_size = data.nEventsPerStat
sprite_length = data.movingWindow
features_tsfresh = data.feature_keys_tsfresh
n_feats_tsfresh = data.n_feats_tsfresh
file_features_tsf = h5py.File(filename_features_tsf,'a')
for ass in data.assets:
    thisAsset = data.AllAssets[str(ass)]
    print(str(ass)+". "+thisAsset)
    # load separators
    separators = load_separators(data, thisAsset, 
                                     separators_directory, 
                                     from_txt=1)
    for s in range(0,len(separators)-1,2):
        
        
        n_events = separators.index[s+1]-separators.index[s]+1
        if n_events>=2*window_size:
            print("\t chuck {0:d} of {1:d}".format(int(s/2),int(len(separators)/2-1))+
                      ". From "+separators.DateTime.iloc[s]+" to "+separators.DateTime.iloc[s+1])

            # init and end dates in string format
            init_date = dt.datetime.strftime(dt.datetime.strptime(
                    separators.DateTime.iloc[s],'%Y.%m.%d %H:%M:%S'),'%y%m%d%H%M%S')
            end_date = dt.datetime.strftime(dt.datetime.strptime(
                    separators.DateTime.iloc[s+1],'%Y.%m.%d %H:%M:%S'),'%y%m%d%H%M%S')
            # hdf5 group name
            group_name_chunck = thisAsset+'/'+init_date+end_date
            for f in features_tsfresh:
                group_name_feat = group_name_chunck+'/'+str(f)
                count = 0
                success = 0
                if group_name_feat in file_features_tsf:
                    if 1:
                    #while count<10 and not success:
                        try:
                            group_chunck = file_features_tsf[group_name_feat]
                            success = 1
                            #print(success)
                        except KeyError:
                            print("KeyError: Unable to open object (Corrupt object header) "+group_name_feat)
                            #del file_features_tsf[group_name_feat]
                            #print(group_name_feat+" deleted")
                            
                            count += 1
                    if count>0:
                        print(group_chunck["feature"])
                else:
                    print("Group "+group_name_feat+" not in file")
                    if group_name_feat not in file_features_tsf:
                        print("Group "+group_name_feat+" not in file")
                
file_features_tsf.close()
print("DONE")   

1. AUDCAD
	 chuck 0 of 21. From 2016.01.04 00:00:08 to 2016.10.13 15:41:59
	 chuck 1 of 21. From 2016.10.13 17:52:00 to 2016.12.23 23:59:59
	 chuck 2 of 21. From 2016.12.26 06:00:00 to 2016.12.30 23:58:30
	 chuck 3 of 21. From 2017.01.02 09:00:00 to 2017.01.26 23:59:59
	 chuck 4 of 21. From 2017.01.30 00:05:00 to 2017.09.05 23:59:59
	 chuck 5 of 21. From 2017.09.06 20:25:01 to 2017.11.10 09:38:15
KeyError: Unable to open object (Corrupt object header) AUDCAD/170906202501171110093815/38
<HDF5 dataset "feature": shape (31806, 1), type "<f8">
	 chuck 6 of 21. From 2017.11.10 10:05:52 to 2017.11.24 23:58:59
	 chuck 7 of 21. From 2017.11.27 00:05:00 to 2017.12.08 23:58:59
	 chuck 8 of 21. From 2017.12.11 00:05:00 to 2017.12.12 04:13:07
	 chuck 9 of 21. From 2017.12.12 09:55:53 to 2017.12.22 23:58:59
	 chuck 10 of 21. From 2017.12.26 03:00:00 to 2017.12.29 23:58:42
	 chuck 11 of 21. From 2018.01.02 03:00:00 to 2018.01.10 21:25:25
	 chuck 12 of 21. From 2018.01.11 14:39:34 to 2018.01.25 08:17

	 chuck 21 of 21. From 2018.08.27 19:03:32 to 2018.11.09 23:59:06
8. EURNZD
	 chuck 0 of 29. From 2016.01.04 00:00:00 to 2016.10.13 15:41:59
	 chuck 1 of 29. From 2016.10.13 17:52:00 to 2016.12.23 23:59:59
	 chuck 2 of 29. From 2016.12.26 06:00:00 to 2016.12.30 23:58:02
	 chuck 3 of 29. From 2017.03.08 00:01:00 to 2017.03.17 22:54:59
	 chuck 4 of 29. From 2017.03.20 01:08:01 to 2017.03.24 22:59:59
	 chuck 5 of 29. From 2017.03.27 01:18:09 to 2017.03.27 23:59:59
	 chuck 6 of 29. From 2017.03.28 02:31:26 to 2017.03.28 23:59:59
	 chuck 7 of 29. From 2017.03.29 02:09:06 to 2017.03.30 23:59:59
	 chuck 8 of 29. From 2017.03.31 02:39:18 to 2017.03.31 23:59:59
	 chuck 9 of 29. From 2017.04.03 02:27:33 to 2017.04.03 23:59:59
	 chuck 10 of 29. From 2017.04.04 02:32:37 to 2017.07.14 00:04:30
	 chuck 11 of 29. From 2017.07.14 00:27:00 to 2017.09.05 02:36:00
	 chuck 13 of 29. From 2017.09.05 09:45:50 to 2017.11.10 09:38:15
	 chuck 14 of 29. From 2017.11.10 10:05:51 to 2017.11.24 23:58:57
	 chuck 15

KeyError: 'Unable to open object (corrupt object header)'

In [ ]:
# Script: Copy extracted tsfresh features to a new file
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from inputs import Data, load_separators
import h5py
import datetime as dt
import time

hdf5_directory = 'D:/SDC/py/HDF5/'
filename_raw = hdf5_directory+'tradeinfo.hdf5'
separators_directory = hdf5_directory+'separators/'
data=Data(movingWindow=100,
              nEventsPerStat=1000,
              dateTest = [                                          '2018.03.09',
                '2018.03.12','2018.03.13','2018.03.14','2018.03.15','2018.03.16',
                '2018.03.19','2018.03.20','2018.03.21','2018.03.22','2018.03.23',
                '2018.03.26','2018.03.27','2018.03.28','2018.03.29','2018.03.30',
                '2018.04.02','2018.04.03','2018.04.04','2018.04.05','2018.04.06',
                '2018.04.09','2018.04.10','2018.04.11','2018.04.12','2018.04.13',
                '2018.04.16','2018.04.17','2018.04.18','2018.04.19','2018.04.20',
                '2018.04.23','2018.04.24','2018.04.25','2018.04.26','2018.04.27',
                '2018.04.30','2018.05.01','2018.05.02','2018.05.03','2018.05.04',
                '2018.05.07','2018.05.08','2018.05.09','2018.05.10','2018.05.11',
                '2018.05.14','2018.05.15','2018.05.16','2018.05.17','2018.05.18',
                '2018.05.21','2018.05.22','2018.05.23','2018.05.24','2018.05.25',
                '2018.05.28','2018.05.29','2018.05.30','2018.05.31','2018.06.01',
                '2018.06.04','2018.06.05','2018.06.06','2018.06.07','2018.06.08',
                '2018.06.11','2018.06.12','2018.06.13','2018.06.14','2018.06.15',
                '2018.06.18','2018.06.19','2018.06.20','2018.06.21','2018.06.22',
                '2018.06.25','2018.06.26','2018.06.27','2018.06.28','2018.06.29',
                '2018.07.02','2018.07.03','2018.07.04','2018.07.05','2018.07.06',
                '2018.07.09','2018.07.10','2018.07.11','2018.07.12','2018.07.13',
                '2018.07.30','2018.07.31','2018.08.01','2018.08.02','2018.08.03',
                '2018.08.06','2018.08.07','2018.08.08','2018.08.09','2018.08.10']+
               ['2018.08.13','2018.08.14','2018.08.15','2018.08.16','2018.08.17',
                '2018.08.20','2018.08.21','2018.08.22','2018.08.23','2018.08.24',
                '2018.08.27','2018.08.28','2018.08.29','2018.08.30','2018.08.31',
                '2018.09.03','2018.09.04','2018.09.05','2018.09.06','2018.09.07',
                '2018.09.10','2018.09.11','2018.09.12','2018.09.13','2018.09.14',
                '2018.09.17','2018.09.18','2018.09.19','2018.09.20','2018.09.21',
                '2018.09.24','2018.09.25','2018.09.26','2018.09.27'],
                 feature_keys_tsfresh=[i for i in range(37,68)])
filename_features_tsf = (hdf5_directory+'feats_tsf_mW'+str(data.movingWindow)+'_nE'+
                         str(data.nEventsPerStat)+'.hdf5')
window_size = data.nEventsPerStat
sprite_length = data.movingWindow
features_tsfresh = data.feature_keys_tsfresh
n_feats_tsfresh = data.n_feats_tsfresh
file_features_tsf = h5py.File(filename_features_tsf,'a')
for ass in data.assets:
    thisAsset = data.AllAssets[str(ass)]
    print(str(ass)+". "+thisAsset)
    # load separators
    separators = load_separators(data, thisAsset, 
                                     separators_directory, 
                                     from_txt=1)
    for s in range(0,len(separators)-1,2):
        
        
        n_events = separators.index[s+1]-separators.index[s]+1
        if n_events>=2*window_size:
            print("\t chuck {0:d} of {1:d}".format(int(s/2),int(len(separators)/2-1))+
                      ". From "+separators.DateTime.iloc[s]+" to "+separators.DateTime.iloc[s+1])

            # init and end dates in string format
            init_date = dt.datetime.strftime(dt.datetime.strptime(
                    separators.DateTime.iloc[s],'%Y.%m.%d %H:%M:%S'),'%y%m%d%H%M%S')
            end_date = dt.datetime.strftime(dt.datetime.strptime(
                    separators.DateTime.iloc[s+1],'%Y.%m.%d %H:%M:%S'),'%y%m%d%H%M%S')
            # hdf5 group name
            group_name_chunck = thisAsset+'/'+init_date+end_date
            for f in features_tsfresh:
                group_name_feat = group_name_chunck+'/'+str(f)
                count = 0
                success = 0
                try:
                    if group_name_feat in file_features_tsf:
                        try:
                            group_chunck = file_features_tsf[group_name_feat]
                            success = 1
                            #print(group_name_feat+" in file_features_tsf")
                        except KeyError:
                            print("KeyError: Unable to open object (Corrupt object header) "+group_name_feat)
                            time.sleep(1)
                    else:
                        print("Group "+group_name_feat+" not in file")
                except KeyError:
                    print("KeyError: Unable to open object (Corrupt object header) "+group_name_feat)
                    time.sleep(1)
            time.sleep(1)
file_features_tsf.close()
print("DONE")   

C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1. AUDCAD
	 chuck 0 of 21. From 2016.01.04 00:00:08 to 2016.10.13 15:41:59
	 chuck 1 of 21. From 2016.10.13 17:52:00 to 2016.12.23 23:59:59
	 chuck 2 of 21. From 2016.12.26 06:00:00 to 2016.12.30 23:58:30
	 chuck 3 of 21. From 2017.01.02 09:00:00 to 2017.01.26 23:59:59
	 chuck 4 of 21. From 2017.01.30 00:05:00 to 2017.09.05 23:59:59
	 chuck 5 of 21. From 2017.09.06 20:25:01 to 2017.11.10 09:38:15
KeyError: Unable to open object (Corrupt object header) AUDCAD/170906202501171110093815/38
	 chuck 6 of 21. From 2017.11.10 10:05:52 to 2017.11.24 23:58:59
	 chuck 7 of 21. From 2017.11.27 00:05:00 to 2017.12.08 23:58:59
	 chuck 8 of 21. From 2017.12.11 00:05:00 to 2017.12.12 04:13:07
	 chuck 9 of 21. From 2017.12.12 09:55:53 to 2017.12.22 23:58:59
	 chuck 10 of 21. From 2017.12.26 03:00:00 to 2017.12.29 23:58:42
	 chuck 11 of 21. From 2018.01.02 03:00:00 to 2018.01.10 21:25:25
	 chuck 12 of 21. From 2018.01.11 14:39:34 to 2018.01.25 08:17:37
	 chuck 13 of 21. From 2018.01.25 08:53:48 to 2018.

8. EURNZD
	 chuck 0 of 29. From 2016.01.04 00:00:00 to 2016.10.13 15:41:59
	 chuck 1 of 29. From 2016.10.13 17:52:00 to 2016.12.23 23:59:59
	 chuck 2 of 29. From 2016.12.26 06:00:00 to 2016.12.30 23:58:02
	 chuck 3 of 29. From 2017.03.08 00:01:00 to 2017.03.17 22:54:59
	 chuck 4 of 29. From 2017.03.20 01:08:01 to 2017.03.24 22:59:59
	 chuck 5 of 29. From 2017.03.27 01:18:09 to 2017.03.27 23:59:59
	 chuck 6 of 29. From 2017.03.28 02:31:26 to 2017.03.28 23:59:59
	 chuck 7 of 29. From 2017.03.29 02:09:06 to 2017.03.30 23:59:59
	 chuck 8 of 29. From 2017.03.31 02:39:18 to 2017.03.31 23:59:59
	 chuck 9 of 29. From 2017.04.03 02:27:33 to 2017.04.03 23:59:59
	 chuck 10 of 29. From 2017.04.04 02:32:37 to 2017.07.14 00:04:30
	 chuck 11 of 29. From 2017.07.14 00:27:00 to 2017.09.05 02:36:00
	 chuck 13 of 29. From 2017.09.05 09:45:50 to 2017.11.10 09:38:15
	 chuck 14 of 29. From 2017.11.10 10:05:51 to 2017.11.24 23:58:57
	 chuck 15 of 29. From 2017.11.27 00:05:00 to 2017.12.08 23:58:59
	 chuck 16

	 chuck 23 of 23. From 2018.10.19 18:09:29 to 2018.11.09 23:59:08
13. GBPCHF
	 chuck 0 of 21. From 2016.01.04 00:00:30 to 2016.10.13 15:41:59
KeyError: Unable to open object (Corrupt object header) GBPCHF/160104000030161013154159/37
	 chuck 1 of 21. From 2016.10.13 17:52:00 to 2016.12.23 23:58:59
	 chuck 2 of 21. From 2016.12.26 06:00:00 to 2017.07.17 18:51:04
	 chuck 3 of 21. From 2017.07.18 08:25:27 to 2017.07.18 23:45:30
	 chuck 4 of 21. From 2017.07.19 02:07:47 to 2017.09.04 23:45:59
	 chuck 5 of 21. From 2017.09.05 10:04:57 to 2017.11.10 09:38:15
	 chuck 6 of 21. From 2017.11.10 10:05:53 to 2017.11.24 23:58:59
	 chuck 7 of 21. From 2017.11.27 00:05:00 to 2017.12.08 23:58:59
	 chuck 8 of 21. From 2017.12.11 00:05:00 to 2017.12.12 04:13:02
	 chuck 9 of 21. From 2017.12.12 10:13:40 to 2017.12.22 23:58:59
	 chuck 10 of 21. From 2017.12.26 03:00:01 to 2017.12.29 23:58:59
	 chuck 11 of 21. From 2018.01.02 03:00:00 to 2018.01.10 21:16:30
	 chuck 12 of 21. From 2018.01.11 14:39:29 to 2018

In [20]:
group_name_feat in file_features_tsf

True

In [13]:
# formatear features file
''''
print(b)
print(batches)
cwt_coeff = [3,9,2,3,11,12,5,4,8,14,7,6,13,10,13,10,5,11,5,13,14,3,1,4,12,6,
                 10,4,7,2,9,8,9,1,12,5,2,1,8,11,4,7,6,2]

file_features_tsf = h5py.File(filename_features_tsf,'a')
for ass in data.assets[:1]:
    thisAsset = data.AllAssets[str(ass)]
    # load separators
    separators = load_separators(data, thisAsset, 
                                     separators_directory, 
                                     from_txt=1)
    for s in range(0,len(separators)-1,2):
        init_date = dt.datetime.strftime(dt.datetime.strptime(
                            separators.DateTime.iloc[s],'%Y.%m.%d %H:%M:%S'),'%y%m%d%H%M%S')
        end_date = dt.datetime.strftime(dt.datetime.strptime(
                            separators.DateTime.iloc[s+1],'%Y.%m.%d %H:%M:%S'),'%y%m%d%H%M%S')
        # hdf5 group name
        group_name_chunck = thisAsset+'/'+init_date+end_date
        for f in features_tsfresh:
            group_name_feat = group_name_chunck+'/'+str(f)
            if group_name_feat in file_features_tsf:
                del file_features_tsf[group_name_feat]
                print(group_name_feat+" deleted")
file_features_tsf.close()

print(features)
print(features.shape)
print(batches)
try:
    ft.close()
except:
    print("ft already closed")
''''


SyntaxError: EOL while scanning string literal (<ipython-input-13-cc92d17afddb>, line 36)

In [79]:
data.assets
file_features_tsf.close()

In [31]:
file_features_tsf = h5py.File(filename_features_tsf,'a')
for ass in data.assets[14:15]:
    for s in range(30,len(separators)-1,2):
        n_events = separators.index[s+1]-separators.index[s]+1
        if n_events>=2*window_size:
            print("\t Getting features from raw data...")
            tic = time.time()
            tic_chunck = tic
            # init and end dates in string format
            init_date = dt.datetime.strftime(dt.datetime.strptime(
                    separators.DateTime.iloc[s],'%Y.%m.%d %H:%M:%S'),'%y%m%d%H%M%S')
            end_date = dt.datetime.strftime(dt.datetime.strptime(
                    separators.DateTime.iloc[s+1],'%Y.%m.%d %H:%M:%S'),'%y%m%d%H%M%S')
            # hdf5 group name
            group_name_chunck = thisAsset+'/'+init_date+end_date
            for f in features_tsfresh:
                group_name_feat = group_name_chunck+'/'+str(f)
                if group_name_feat in file_features_tsf:
                    print(group_name_feat+" deleted")
                    del file_features_tsf[group_name_feat]
file_features_tsf.close()

	 Getting features from raw data...
	 Getting features from raw data...
USDJPY/171226030000171229235859/41 deleted
USDJPY/171226030000171229235859/42 deleted
USDJPY/171226030000171229235859/43 deleted
USDJPY/171226030000171229235859/44 deleted
USDJPY/171226030000171229235859/45 deleted
USDJPY/171226030000171229235859/46 deleted
USDJPY/171226030000171229235859/47 deleted
USDJPY/171226030000171229235859/48 deleted
USDJPY/171226030000171229235859/49 deleted
USDJPY/171226030000171229235859/50 deleted
USDJPY/171226030000171229235859/51 deleted
USDJPY/171226030000171229235859/52 deleted
USDJPY/171226030000171229235859/53 deleted
USDJPY/171226030000171229235859/54 deleted
USDJPY/171226030000171229235859/55 deleted
USDJPY/171226030000171229235859/56 deleted
USDJPY/171226030000171229235859/57 deleted
USDJPY/171226030000171229235859/58 deleted
USDJPY/171226030000171229235859/59 deleted
USDJPY/171226030000171229235859/60 deleted
USDJPY/171226030000171229235859/61 deleted
USDJPY/171226030000171229

In [23]:
c = 0
# save features in HDF5 file
for f in features_tsfresh:
    group_name_feat = group_name_chunck+'/'+str(f)
    params = data.AllFeatures[str(f)]
    n_new_feats = params[-1]
    #if group_name_feat in file_features_tsf:
    #    del file_features_tsf[group_name_feat]
    if group_name_feat not in file_features_tsf:
        group_chunck = file_features_tsf.create_group(group_name_feat)
        group_features = group_chunck.create_dataset("feature", (m,n_new_feats),dtype=float)
        group_features[:,:] = features[:,c:c+n_new_feats]
    else: # load features from HDF5 file if they are saved already
        params = data.AllFeatures[str(f)]
        n_new_feats = params[-1]
        group_chunck = file_features_tsf[group_name_feat]
        features[:,c:c+n_new_feats] = group_chunck["feature"]
        c += n_new_feats

In [42]:
# extract most common filtered features from csv iles
import pandas as pd
from inputs import Data,load_separators
import os

hdf5_directory = 'D:/SDC/py/HDF5/'
separators_directory = hdf5_directory+'separators/'
file_extension = '.csv'
data = Data(movingWindow=100,nEventsPerStat=1000)
# init scores of most common features
feature_scores = pd.DataFrame(columns=['features','scores'])
counter = 0
# loop over assets
for ass in data.assets:
    thisAsset = data.AllAssets[str(ass)]
    print(str(ass)+". "+thisAsset)
    # load separators
    separators = load_separators(data, thisAsset, separators_directory, from_txt=1)
    # loop over chuncks
    for s in range(0,len(separators)-1,2):
        filename = thisAsset+'c'+str(int(s/2))
        # read csv file
        if os.path.exists('../features/'+filename+file_extension):
            counter += 1
            this_df = pd.read_csv('../features/'+filename+file_extension)
            print(filename+" # features "+str(this_df.shape[1]))
            # increase scores
            for c in this_df.columns[1:]:
                if (feature_scores.features.str.find(c)==0).sum()>0:
                    feature_scores.scores[feature_scores.features==c] += 1
                else:
                    feature_scores = feature_scores.append(pd.DataFrame({'features':c,'scores':1},index=[0]),ignore_index=True)
print(feature_scores.shape)
#feature_scores.scores.sort_values(ascending=False)
feature_scores_ordered = feature_scores.loc[feature_scores.scores.sort_values(ascending=False).index]

max_score = feature_scores.scores.max()
reps_max_score = (feature_scores.scores==max_score).sum()
print("Max possible score="+str(counter))
print(feature_scores_ordered.iloc[:reps_max_score])
cum_sum = 0
for sc in range(max_score,0,-1):
    tsum = (feature_scores.scores==sc).sum()
    print(str(sc)+" score: "+str(tsum)+" times")
    cum_sum += tsum
    print("Cummulated sum: "+str(cum_sum))
last_r="""
24 score: 14 times
23 score: 48 times
22 score: 13 times
21 score: 0 times
20 score: 0 times
19 score: 1 times
18 score: 0 times
17 score: 0 times
16 score: 0 times
15 score: 3 times
14 score: 3 times
13 score: 16 times
12 score: 86 times
11 score: 35 times
10 score: 51 times
9 score: 63 times
8 score: 42 times
7 score: 36 times
6 score: 44 times
5 score: 57 times
4 score: 42 times
3 score: 34 times
2 score: 39 times
1 score: 37 times
"""

1. AUDCAD
AUDCADc0 # features 375
AUDCADc1 # features 130
AUDCADc2 # features 426
AUDCADc3 # features 422
AUDCADc4 # features 276
AUDCADc5 # features 382
AUDCADc6 # features 109
AUDCADc7 # features 330
AUDCADc8 # features 128
AUDCADc9 # features 110
AUDCADc10 # features 101
AUDCADc11 # features 104
AUDCADc12 # features 173
AUDCADc13 # features 267
AUDCADc14 # features 122
AUDCADc15 # features 115
AUDCADc16 # features 405
AUDCADc17 # features 297
AUDCADc18 # features 117
AUDCADc19 # features 105
AUDCADc20 # features 7
AUDCADc21 # features 110
2. EURAUD
EURAUDc0 # features 111
EURAUDc1 # features 308
EURAUDc2 # features 406
EURAUDc3 # features 16
EURAUDc4 # features 235
EURAUDc5 # features 441
EURAUDc6 # features 229
EURAUDc7 # features 101
EURAUDc9 # features 227
EURAUDc10 # features 448
EURAUDc11 # features 38
EURAUDc12 # features 385
EURAUDc15 # features 373
EURAUDc16 # features 107
EURAUDc17 # features 93
EURAUDc18 # features 459
EURAUDc19 # features 312
EURAUDc20 # features 216
EURA

In [27]:
data.assets

[1, 2, 3, 4, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 19, 27, 28, 29, 30, 31, 32]

In [21]:
import tsfresh
x=input_ts.SymbolBid
param = {"coeff": 0, "attr": "real"}
#print(param)

print('SymbolBid__fft_coefficient__coeff_0__attr_"real"')
print(fft_coefficient(x, [0, "real"]))
print(features['SymbolBid__fft_coefficient__coeff_0__attr_"real"'].iloc[-1])

print('SymbolBid__linear_trend__attr_"slope"')
print(linear_trend(x, "slope"))
print(features['SymbolBid__linear_trend__attr_"slope"'].iloc[-1])

print('SymbolBid__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"intercept"')
print(agg_linear_trend(x, ["min",50,"intercept"]))
print(features['SymbolBid__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"intercept"'].iloc[-1])

print('SymbolBid__first_location_of_minimum')
print(first_location_of_minimum(x))
print(features['SymbolBid__first_location_of_minimum'].iloc[-1])

print('SymbolBid__energy_ratio_by_chunks__num_segments_10__segment_focus_1')
print(energy_ratio_by_chunks(x, 10, 1))
print(features['SymbolBid__energy_ratio_by_chunks__num_segments_10__segment_focus_1'].iloc[-1])

print('SymbolBid__change_quantiles__f_agg_"mean"__isabs_False__qh_0.2__ql_0.0')
print(change_quantiles(x, 0.0, 0.2, False, "mean"))
print(features['SymbolBid__change_quantiles__f_agg_"mean"__isabs_False__qh_0.2__ql_0.0'].iloc[-1])
#print(feature_scores_ordered.features.loc[357])

print('SymbolBid__index_mass_quantile__q_0.9')
print(index_mass_quantile(x, 0.9))
print(features['SymbolBid__index_mass_quantile__q_0.9'].iloc[-1])

print('SymbolBid__number_cwt_peaks__n_5')
print(number_cwt_peaks(x, 5))
print(features['SymbolBid__number_cwt_peaks__n_5'].iloc[-1])

print('SymbolBid__last_location_of_maximum')
print(last_location_of_maximum(x))
print(features['SymbolBid__last_location_of_maximum'].iloc[-1])

print('SymbolBid__first_location_of_maximum')
print(first_location_of_maximum(x))
print(features['SymbolBid__first_location_of_maximum'].iloc[-1])

print('SymbolBid__last_location_of_minimum')
print(last_location_of_minimum(x))
print(features['SymbolBid__last_location_of_minimum'].iloc[-1])

print('SymbolBid__mean_change')
print(mean_change(x))
print(features['SymbolBid__mean_change'].iloc[-1])

print('SymbolBid__sum_values')
print(sum_values(x,[]))
print(features['SymbolBid__sum_values'].iloc[-1])

print('SymbolBid__mean')
print(mean(x,[]))
print(features['SymbolBid__mean'].iloc[-1])

print('SymbolBid__minimum')
print(minimum(x,[]))
print(features['SymbolBid__minimum'].iloc[-1])

print('SymbolBid__median')
print(median(x,[]))
print(features['SymbolBid__median'].iloc[-1])

print('SymbolBid__c3__lag_3')
print(c3(x, [3]))
print(features['SymbolBid__c3__lag_3'].iloc[-1])

print('SymbolBid__quantile__q_0.3')
print(quantile(x, [0.3]))
print(features['SymbolBid__quantile__q_0.3'].iloc[-1])

print('SymbolBid__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_3__w_5')
print(cwt_coefficients(x, [[2, 5, 10, 20],[3,9,2,3,11,12,5,4,8,14],[5,10,2,2,10,10,5,5,5,10]]))
print(features['SymbolBid__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_3__w_5'].iloc[-1])

print('SymbolBid__maximum')
print(maximum(x,[]))
print(features['SymbolBid__maximum'].iloc[-1])

print('SymbolBid__abs_energy')
print(abs_energy(x, []))
print(features['SymbolBid__abs_energy'].iloc[-1])

print('SymbolBid__linear_trend__attr_"intercept"')
print(linear_trend(x, "intercept"))
print(features['SymbolBid__linear_trend__attr_"intercept"'].iloc[-1])
#print(feature_scores.scores[feature_scores_ordered.features=='SymbolBid__fft_coefficient__coeff_0__attr_"real"'])
#print(features.SymbolBid__last_location_of_maximum)

SymbolBid__fft_coefficient__coeff_0__attr_"real"
1008.04491
1008.04491
SymbolBid__linear_trend__attr_"slope"
3.0449038649e-06
3.0449038649e-06
SymbolBid__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"intercept"
1.00643128571
1.00643128571
SymbolBid__first_location_of_minimum
1.247
0.247
SymbolBid__energy_ratio_by_chunks__num_segments_10__segment_focus_1
0.099895855316
0.099895855316
SymbolBid__change_quantiles__f_agg_"mean"__isabs_False__qh_0.2__ql_0.0
1.95767195767e-06
1.95767195767e-06
SymbolBid__index_mass_quantile__q_0.9
0.901
0.901
SymbolBid__number_cwt_peaks__n_5
98
98.0
SymbolBid__last_location_of_maximum
0.899
0.899
SymbolBid__first_location_of_maximum
1.898
0.898
SymbolBid__last_location_of_minimum
0.248
0.248
SymbolBid__mean_change
3.25325325325e-06
3.25325325325e-06
SymbolBid__sum_values
1008.04491
1008.04491
SymbolBid__mean
1.00804491
1.00804491
SymbolBid__minimum
1.00671
1.00671
SymbolBid__median
1.0078
1.0078
SymbolBid__c3__lag_3
1.02432302731
1.02432302731
SymbolBid

In [4]:
from scipy.stats import linregress
from scipy.signal import cwt, find_peaks_cwt, ricker, welch

def complex_agg(x, agg):
    if agg == "real":
        return x.real
    elif agg == "imag":
        return x.imag
    elif agg == "abs":
        return np.abs(x)
    elif agg == "angle":
        return np.angle(x, deg=True)

def _aggregate_on_chunks(x, f_agg, chunk_len):
    """
    Takes the time series x and constructs a lower sampled version of it by applying the aggregation function f_agg on
    consecutive chunks of length chunk_len

    :param x: the time series to calculate the aggregation of
    :type x: pandas.Series
    :param f_agg: The name of the aggregation function that should be an attribute of the pandas.Series
    :type f_agg: str
    :param chunk_len: The size of the chunks where to aggregate the time series
    :type chunk_len: int
    :return: A list of the aggregation function over the chunks
    :return type: list
    """
    return [getattr(x[i * chunk_len: (i + 1) * chunk_len], f_agg)() for i in range(int(np.ceil(len(x) / chunk_len)))]    

def quantile(x, param):
    """
    Calculates the q quantile of x. This is the value of x greater than q% of the ordered values from x.

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :param q: the quantile to calculate
    :type q: float
    :return: the value of this feature
    :return type: float
    """
    #x = pd.Series(x)
    return np.percentile(x, 100*param[0])

def fft_coefficient(x, param):
    """
    Calculates the fourier coefficients of the one-dimensional discrete Fourier Transform for real input by fast
    fourier transformation algorithm

    .. math::
        A_k =  \\sum_{m=0}^{n-1} a_m \\exp \\left \\{ -2 \\pi i \\frac{m k}{n} \\right \\}, \\qquad k = 0,
        \\ldots , n-1.

    The resulting coefficients will be complex, this feature calculator can return the real part (attr=="real"),
    the imaginary part (attr=="imag), the absolute value (attr=""abs) and the angle in degrees (attr=="angle).

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :param param: contains dictionaries {"coeff": x, "attr": s} with x int and x >= 0, s str and in ["real", "imag",
        "abs", "angle"]
    :type param: list
    :return: the different feature values
    :return type: pandas.Series
    """

    #assert min([config["coeff"] for config in param]) >= 0, "Coefficients must be positive or zero."
    #assert set([config["attr"] for config in param]) <= set(["imag", "real", "abs", "angle"]), \
    #    'Attribute must be "real", "imag", "angle" or "abs"'
    
    fft = np.fft.rfft(x)

    

    res = complex_agg(fft[param[0]], param[1])
    return res

def linear_trend(x, param):
    """
    Calculate a linear least-squares regression for the values of the time series versus the sequence from 0 to
    length of the time series minus one.
    This feature assumes the signal to be uniformly sampled. It will not use the time stamps to fit the model.
    The parameters control which of the characteristics are returned.

    Possible extracted attributes are "pvalue", "rvalue", "intercept", "slope", "stderr", see the documentation of
    linregress for more information.

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :param param: contains dictionaries {"attr": x} with x an string, the attribute name of the regression model
    :type param: list
    :return: the different feature values
    :return type: pandas.Series
    """
    # todo: we could use the index of the DataFrame here
    attr = param[0]
    linReg = linregress(range(len(x)), x)

    return getattr(linReg, attr)

def agg_linear_trend(x, param):
    """
    Calculates a linear least-squares regression for values of the time series that were aggregated over chunks versus
    the sequence from 0 up to the number of chunks minus one.

    This feature assumes the signal to be uniformly sampled. It will not use the time stamps to fit the model.

    The parameters attr controls which of the characteristics are returned. Possible extracted attributes are "pvalue",
    "rvalue", "intercept", "slope", "stderr", see the documentation of linregress for more information.

    The chunksize is regulated by "chunk_len". It specifies how many time series values are in each chunk.

    Further, the aggregation function is controlled by "f_agg", which can use "max", "min" or , "mean", "median"

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :param param: contains dictionaries {"attr": x, "chunk_len": l, "f_agg": f} with x, f an string and l an int
    :type param: list
    :return: the different feature values
    :return type: pandas.Series
    """
    # todo: we could use the index of the DataFrame here
    calculated_agg = {}
    f_agg = param[0]
    chunk_len = param[1]
    attr = param[2]
    
    aggregate_result = _aggregate_on_chunks(x, f_agg, chunk_len)
    if f_agg not in calculated_agg or chunk_len not in calculated_agg[f_agg]:
        if chunk_len >= len(x):
            calculated_agg[f_agg] = {chunk_len: np.NaN}
        else:
            lin_reg_result = linregress(range(len(aggregate_result)), aggregate_result)
            calculated_agg[f_agg] = {chunk_len: lin_reg_result}

    if chunk_len >= len(x):
        res_data = np.NaN
    else:
        res_data = getattr(calculated_agg[f_agg][chunk_len], attr)
        
    return res_data

def first_location_of_minimum(x):
    """
    Returns the first location of the minimal value of x.
    The position is calculated relatively to the length of x.

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :return: the value of this feature
    :return type: float
    """
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    return np.argmin(x) / len(x) if len(x) > 0 else np.NaN

def energy_ratio_by_chunks(x, num_segments, segment_focus):
    """
    Calculates the sum of squares of chunk i out of N chunks expressed as a ratio with the sum of squares over the whole
    series.

    Takes as input parameters the number num_segments of segments to divide the series into and segment_focus
    which is the segment number (starting at zero) to return a feature on.

    If the length of the time series is not a multiple of the number of segments, the remaining data points are
    distributed on the bins starting from the first. For example, if your time series consists of 8 entries, the
    first two bins will contain 3 and the last two values, e.g. `[ 0.,  1.,  2.], [ 3.,  4.,  5.]` and `[ 6.,  7.]`.

    Note that the answer for `num_segments = 1` is a trivial "1" but we handle this scenario
    in case somebody calls it. Sum of the ratios should be 1.0.

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :param param: contains dictionaries {"num_segments": N, "segment_focus": i} with N, i both ints
    :return: the feature values
    :return type: list of tuples (index, data)
    """
    res_data = []
    res_index = []
    full_series_energy = np.sum(x ** 2)

    assert segment_focus < num_segments
    assert num_segments > 0

    res_data = np.sum(np.array_split(x, num_segments)[segment_focus] ** 2.0)/full_series_energy

    return res_data

def change_quantiles(x, ql, qh, isabs, f_agg):
    """
    First fixes a corridor given by the quantiles ql and qh of the distribution of x.
    Then calculates the average, absolute value of consecutive changes of the series x inside this corridor.

    Think about selecting a corridor on the
    y-Axis and only calculating the mean of the absolute change of the time series inside this corridor.

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :param ql: the lower quantile of the corridor
    :type ql: float
    :param qh: the higher quantile of the corridor
    :type qh: float
    :param isabs: should the absolute differences be taken?
    :type isabs: bool
    :param f_agg: the aggregator function that is applied to the differences in the bin
    :type f_agg: str, name of a numpy function (e.g. mean, var, std, median)

    :return: the value of this feature
    :return type: float
    """
    if ql >= qh:
        ValueError("ql={} should be lower than qh={}".format(ql, qh))

    div = np.diff(x)
    if isabs:
        div = np.abs(div)
    # All values that originate from the corridor between the quantiles ql and qh will have the category 0,
    # other will be np.NaN
    try:
        bin_cat = pd.qcut(x, [ql, qh], labels=False)
        bin_cat_0 = bin_cat == 0
    except ValueError:  # Occurs when ql are qh effectively equal, e.g. x is not long enough or is too categorical
        return 0
    # We only count changes that start and end inside the corridor
    ind = (bin_cat_0 & np.roll(bin_cat_0, 1))[1:]
    if sum(ind) == 0:
        return 0
    else:
        ind_inside_corridor = np.where(ind == 1)
        aggregator = getattr(np, f_agg)
        return aggregator(div[ind_inside_corridor])
    
def index_mass_quantile(x, q):
    """
    Those apply features calculate the relative index i where q% of the mass of the time series x lie left of i.
    For example for q = 50% this feature calculator will return the mass center of the time series

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :param param: contains dictionaries {"q": x} with x float
    :type param: list
    :return: the different feature values
    :return type: pandas.Series
    """

    x = np.asarray(x)
    abs_x = np.abs(x)
    s = sum(abs_x)

    if s == 0:
        # all values in x are zero or it has length 0
        return np.NaN
    else:
        # at least one value is not zero
        mass_centralized = np.cumsum(abs_x) / s
        return (np.argmax(mass_centralized >= q)+1)/len(x)
    
def number_cwt_peaks(x, n):
    """
    This feature calculator searches for different peaks in x. To do so, x is smoothed by a ricker wavelet and for
    widths ranging from 1 to n. This feature calculator returns the number of peaks that occur at enough width scales
    and with sufficiently high Signal-to-Noise-Ratio (SNR)

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :param n: maximum width to consider
    :type n: int
    :return: the value of this feature
    :return type: int
    """
    return len(find_peaks_cwt(vector=x, widths=np.array(list(range(1, n + 1))), wavelet=ricker))

def last_location_of_maximum(x):
    """
    Returns the relative last location of the maximum value of x.
    The position is calculated relatively to the length of x.

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :return: the value of this feature
    :return type: float
    """
    x = np.asarray(x)
    return 1.0 - np.argmax(x[::-1]) / len(x) if len(x) > 0 else np.NaN

def first_location_of_maximum(x):
    """
    Returns the first location of the maximum value of x.
    The position is calculated relatively to the length of x.

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :return: the value of this feature
    :return type: float
    """
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    return np.argmax(x) / len(x) if len(x) > 0 else np.NaN

def last_location_of_minimum(x):
    """
    Returns the last location of the minimal value of x.
    The position is calculated relatively to the length of x.

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :return: the value of this feature
    :return type: float
    """
    x = np.asarray(x)
    return 1.0 - np.argmin(x[::-1]) / len(x) if len(x) > 0 else np.NaN

def mean_change(x):
    """
    Returns the mean over the absolute differences between subsequent time series values which is

    .. math::

        \\frac{1}{n} \\sum_{i=1,\ldots, n-1}  x_{i+1} - x_{i}

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :return: the value of this feature
    :return type: float
    """
    return np.mean(np.diff(x))

def abs_energy(x, param):
    """
    Returns the absolute energy of the time series which is the sum over the squared values

    .. math::

        E = \\sum_{i=1,\ldots, n} x_i^2

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :return: the value of this feature
    :return type: float
    """
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    return np.dot(x, x)

def sum_values(x, param):
    """
    Calculates the sum over the time series values

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :return: the value of this feature
    :return type: bool
    """
    if len(x) == 0:
        return 0

    return np.sum(x)

def mean(x, param):
    """
    Returns the mean of x

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :return: the value of this feature
    :return type: float
    """
    return np.mean(x)

def minimum(x, param):
    """
    Calculates the lowest value of the time series x.

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :return: the value of this feature
    :return type: float
    """
    return np.min(x)

def median(x, param):
    """
    Returns the median of x

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :return: the value of this feature
    :return type: float
    """
    return np.median(x)

def c3(x, param):
    """
    This function calculates the value of

    .. math::

        \\frac{1}{n-2lag} \sum_{i=0}^{n-2lag} x_{i + 2 \cdot lag}^2 \cdot x_{i + lag} \cdot x_{i}

    which is

    .. math::

        \\mathbb{E}[L^2(X)^2 \cdot L(X) \cdot X]

    where :math:`\\mathbb{E}` is the mean and :math:`L` is the lag operator. It was proposed in [1] as a measure of
    non linearity in the time series.

    .. rubric:: References

    |  [1] Schreiber, T. and Schmitz, A. (1997).
    |  Discrimination power of measures for nonlinearity in a time series
    |  PHYSICAL REVIEW E, VOLUME 55, NUMBER 5

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :param lag: the lag that should be used in the calculation of the feature
    :type lag: int
    :return: the value of this feature
    :return type: float
    """
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    n = x.size
    if 2 * param[0] >= n:
        return 0
    else:
        return np.mean((np.roll(x, 2 * -param[0]) * np.roll(x, -param[0]) * x)[0:(n - 2 * param[0])])
    
def cwt_coefficients(x, param):
    """
    Calculates a Continuous wavelet transform for the Ricker wavelet, also known as the "Mexican hat wavelet" which is
    defined by

    .. math::
        \\frac{2}{\\sqrt{3a} \\pi^{\\frac{1}{4}}} (1 - \\frac{x^2}{a^2}) exp(-\\frac{x^2}{2a^2})

    where :math:`a` is the width parameter of the wavelet function.

    This feature calculator takes three different parameter: widths, coeff and w. The feature calculater takes all the
    different widths arrays and then calculates the cwt one time for each different width array. Then the values for the
    different coefficient for coeff and width w are returned. (For each dic in param one feature is returned)

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :param param: contains dictionaries {"widths":x, "coeff": y, "w": z} with x array of int and y,z int
    :type param: list
    :return: the different feature values
    :return type: pandas.Series
    """
    
    calculated_cwt = {}
    widths = param[0]
    coeff = param[1]
    w = param[2]
    res = np.zeros((len(w)))
    #print(coeff)
    calculated_cwt_for_widths = cwt(x, ricker, widths)
    #print(calculated_cwt_for_widths)
    for r in range(len(w)):
        i = widths.index(w[r])
        if calculated_cwt_for_widths.shape[1] <= coeff[r]:
            res[r] = np.NaN
        else:
            res[r] = calculated_cwt_for_widths[i, coeff[r]]

    return res

def maximum(x,param):
    """
    Calculates the highest value of the time series x.

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :return: the value of this feature
    :return type: float
    """
    return np.max(x)

In [ ]:
a=1

In [9]:
feature_scores_ordered.to_csv('../features/feature_scores_ordered.csv')

In [70]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [71]:
#feature_scores = pd.DataFrame(columns=['features','scores'])
feature_scores = feature_scores.append(pd.DataFrame({'features':c,'scores':1},index=[0]),ignore_index=True)

feature_scores.scores[feature_scores.features==c] += 1
print(feature_scores)

                                            features scores
0  SymbolBid__fft_coefficient__coeff_30__attr_"an...      4
1  SymbolBid__fft_coefficient__coeff_30__attr_"an...      3
2  SymbolBid__fft_coefficient__coeff_30__attr_"an...      2


In [167]:
import tsfresh
print(tsfresh.feature_extraction.feature_calculators.number_cwt_peaks(x, 5))
param=[{"f_agg": "mean", "maxlag": 4}]
tsfresh.feature_extraction.feature_calculators.agg_autocorrelation(x,param)

98


[('f_agg_"mean"', 0.92831114415939009)]

In [40]:
from config import print_config, modify_config
print_config('C0286T')

{'IDweights': '000286', 'assets': [1, 2, 3, 4, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 19, 27, 28, 29, 30, 31, 32], 'nEventsPerStat': 1000, 'outputGain': 0.6, 'dateTest': ['2018.03.09', '2018.03.12', '2018.03.13', '2018.03.14', '2018.03.15', '2018.03.16', '2018.03.19', '2018.03.20', '2018.03.21', '2018.03.22', '2018.03.23', '2018.03.26', '2018.03.27', '2018.03.28', '2018.03.29', '2018.03.30', '2018.04.02', '2018.04.03', '2018.04.04', '2018.04.05', '2018.04.06', '2018.04.09', '2018.04.10', '2018.04.11', '2018.04.12', '2018.04.13', '2018.04.16', '2018.04.17', '2018.04.18', '2018.04.19', '2018.04.20', '2018.04.23', '2018.04.24', '2018.04.25', '2018.04.26', '2018.04.27', '2018.04.30', '2018.05.01', '2018.05.02', '2018.05.03', '2018.05.04', '2018.05.07', '2018.05.08', '2018.05.09', '2018.05.10', '2018.05.11', '2018.05.14', '2018.05.15', '2018.05.16', '2018.05.17', '2018.05.18', '2018.05.21', '2018.05.22', '2018.05.23', '2018.05.24', '2018.05.25', '2018.05.28', '2018.05.29', '2018.05.30', '201

True

In [ ]:
#modify_config('C0286','hdf5_directory','D:/SDC/py/HDF5/')
modify_config('C0286T','save_journal',True)
#modify_config('C0286','endAt',6)
#modify_config('C0286','startFrom',6)
modify_config('C0286T','if_build_IO',False)

0

'SymbolBid__fft_coefficient__coeff_30__attr_"angle"'